## Illustrating the LSC EvoNetV1Dataset


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import torch
torch.cuda.is_available()
# torch.cuda.device_count()

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np


from evotrain.models.lsc10.v16.dataset import EvoNetV1Dataset

In [ ]:
ds = EvoNetV1Dataset(
    bands=["B04", "B03", "B02", "cop-DEM-alt", "VV", "VH", 'VHonVV'],#"VHVV-RATIO"],
    rgb_bands=None,
    flip_augmentation=True,
    rotate_augmentation=True,
    augmented_scaling=True,
    k_factor=5,
    k_factor_jitter=2,
    meteo_jitter=0.05,
    latlon_jitter=0.5,
    doy_jitter=15,
    shuffle_locs=False,
    sort_locs_by_latitude=False,
)

In [ ]:
# ds[8601]

In [ ]:
tst = ds[38101] #ds[86011] #ds[8601]

In [ ]:
tst[0].shape

In [ ]:
len(tst[-1][-1])
(tst[-1][-1].numpy()[-1,...].squeeze()).min()

In [ ]:
# tst[0].numpy().squeeze()[3,...].max()
plt.imshow(tst[-1][-1].numpy()[-1,...].squeeze())

In [ ]:
np.nanmax(tst[0].numpy().squeeze()[2,...])

In [ ]:
np.nanmax(tst[0].numpy().squeeze()[6,...])

In [ ]:
rgb = tst[0].numpy().squeeze()[6,...]
rgb[rgb == -1] = np.nan
plt.imshow(rgb)

In [ ]:
rgb = tst[0].numpy().squeeze()[:3,...]
rgb = np.moveaxis(rgb, 0, -1)
rgb[rgb == -1] = np.nan
plt.imshow(rgb*3.5)

In [ ]:
# ds[2]
quarter_ids = ds.ids
quarter_ids.index('11WMM_078_25_2020Q3')

In [ ]:
import xarray as xr 


In [ ]:
sample_id = '11WMM_078_25_20200720'
ds.read_feats(sample_id, ["VHonVV"])


In [ ]:
sample_id = '11WMM_078_25_20200720'
darr = ds.read_feats(sample_id, ["B04", "B03", "B02", "VV", "VH", "VHonVV"])
# ids = ds.ids
# # arr_s2 = ds.evo_dataset._reader()._read_s2('11WMM_078_25_20200214')
# ids.index(sample_id)

In [ ]:
feats_bands = darr.band.data.tolist()

from evotrain.v2tsm.bands import BANDS_V2TSM_S1_GAMMA0

[b for b in BANDS_V2TSM_S1_GAMMA0 if b in feats_bands]


In [ ]:
darr.loc[dict(band="VV")] = -1
darr.loc[dict(band="VH")] = -1
darr.loc[dict(band="VHonVV")] = -1

In [ ]:
darr

In [ ]:
darr.loc[dict(band="VV")]

In [ ]:
['VV'] in darr.band.data.tolist()
any(map(lambda v: v in darr.band.data.tolist(), ["VV", "VH", "VHonVV"]))
# darr.band.data.tolist()

In [ ]:
darr

In [ ]:
import random
random.randrange(0,1000)

In [ ]:
import matplotlib.pyplot as plt
rgb = darr.sel(band = ["B04", "B03", "B02"]).data
rgb = np.moveaxis(rgb,0,-1)
plt.imshow(rgb*4.5)

In [ ]:
rgb = darr.sel(band = ["VH"]).data
rgb = rgb.squeeze()
plt.imshow(rgb)

In [ ]:
from evotrain.v2tsm import EvoTrainV2TSMDataset
evo_ds = EvoTrainV2TSMDataset(
        # dataset_path=dataset_path
    )

In [ ]:
tmp = ds.evo_dataset.locs.copy()
tmp[tmp['location_id'] == '11WMM_078_25']['sample_id']

In [ ]:

# feats, feats_head, probs, weights = ds[it]

In [ ]:
import xarray as xr
pth = '/projects/TAP/vegteam/training_data/GAMMA0-RAW/v001/patches/11/W/MM/2020/LCFM_GAMMA0-RAW_2020_11WMM_078_25_V001.nc'

darr = xr.open_dataset(pth)


In [ ]:
darr.t.data

In [ ]:
import satio_pc
from satio_pc.extension import SatioTimeSeries
# vv = darr['VV']
# bounds = vv.satio.bounds

In [ ]:
bounds = [int(b) for b in list(bounds)]

In [ ]:
bounds

In [ ]:
vv.shape

In [ ]:
# # epsg = 4326 #src.crs.to_epsg()
# from satio_pc.geotiff import compute_pixel_coordinates
# attrs = {"bounds": bounds}#"epsg": epsg, 
# bands = ["VV"]
# time = list(vv.t.data)
# new_y, new_x = compute_pixel_coordinates(bounds, vv.shape[-2:])

# darr = xr.DataArray(
#     vv.expand_dims({'band': 1}, axis=1).data,
#     dims=["time", "band", "y", "x"],
#     coords={"time": time, "band": bands, "y": new_y, "x": new_x},
#     attrs=attrs,
# )

In [ ]:
# darr.satio.rescale(scale=2, order=0)

In [ ]:

# isinstance(new_res, xr.DataArray)

In [ ]:
# int(new_res / 2)

In [ ]:
# import numpy as np
# scale = 2
# xmin, ymin, xmax, ymax = darr.satio.bounds
# new_res = darr.attrs.get('resolution',
#                        vv.x[1] - vv.x[0]) / scale
# new_res_half = new_res.data / 2

# old_x = vv.x.size
# old_y = vv.y.size

# new_x = np.linspace(xmin + new_res_half,
#                     xmax - new_res_half,
                    # int(old_x * scale))

In [ ]:
tmp = ds.evo_dataset.locs.copy()
tmp[tmp['location_id'] == '11WMM_078_25']['sample_id']

In [ ]:
# vv = darr.VV
# vv
# darr

In [ ]:
# Rename the dimensions using rename_dims
s1_bands = ['VV', 'VH', 'VHonVV']
list_darrs = []
for band in s1_bands:
    vv = darr[band]
    vv = vv.rename({"t": "time"})
    vv = vv.expand_dims({'band': 1}, axis=1)
    vv = vv.assign_coords(band=[band])
    list_darrs.append(vv)
darr_all = xr.concat(
            list_darrs,
            dim="band",
            compat="equals",
        )


from satio_pc.extension import SatioTimeSeries
darr_all = darr_all.satio.rescale(scale=2, order=0)

In [ ]:
darr_all.shape

In [ ]:
darr_all

In [ ]:
# arr = vv.sel(time = vv.time.data[0]).data.squeeze()
# arr.shape

In [ ]:
# vv
# vv.x.size

In [ ]:
# vv.satio.bounds

In [ ]:
# vv.attrs.get('resolution',
#                              vv.x[1] - vv.x[0])

In [ ]:
# xmin + new_res_half

In [ ]:
# blurred_scl20 = xr.DataArray(
#     blurred_scl20, dims=["time", "band", "y", "x"]
# )
# blurred_scl10 = blurred_scl20.satio.rescale(scale=2, order=0)

In [ ]:
# import xarray as xr
# from satio_pc.extension import SatioTimeSeries
# # import satio_pc
# # import satio
# blurred_scl20 = xr.DataArray(
#         np.ones((4,5,64,64)), dims=["time", "band", "y", "x"]
#     )
# blurred_scl20.shape
# blurred_scl20 = blurred_scl20.transpose("time", "band", "y", "x")
    

In [ ]:
# blurred_scl20.satio.rescale(scale=2, order=0)

In [ ]:
# vv['x'] = vv.x.astype(np.int64)

In [ ]:
# vv.x

In [ ]:
# vv.x[1] - vv.x[0]

In [ ]:
# scale = 2
# xmin, ymin, xmax, ymax = vv.satio.bounds
# new_res = vv.attrs.get('resolution',
#                        vv.x[1] - vv.x[0]) / scale
# new_res_half = int(new_res / 2)

# old_x = vv.x.size
# old_y = vv.y.size

# new_x = np.linspace(xmin + new_res_half,
#                     xmax - new_res_half,
#                     int(old_x * scale))

In [ ]:
# xmin + new_res_half

In [ ]:
# xmax - new_res_half

In [ ]:
# int(old_x * scale)

In [ ]:
# new_res_half

In [ ]:
# np.linspace(int(xmin) + int(new_res_half),
#             int(xmax) - new_res_half,
#                     int(old_x * scale))

In [ ]:
vv.data.shape

In [ ]:
arr_s2

In [ ]:
# import satio_pc
# vv = darr.VV
# vv = vv.rename({"t": "time"})

# vv = vv.expand_dims({'band': 1}, axis=1)
# vv = vv.assign_coords(band=["VV"])
# # vv = vv.sel(t = vv.t.data[1])
# vv.satio.rescale(scale=2, order=0)
year = '2020'
month = '02'
day = '01'
np.datetime64(f'{year}-{month}-{day}')

In [ ]:
np.datetime64(f'{year}-{month}-{day}') in list(darr_all.time.data)

In [ ]:
# darr_all.sel(time = np.datetime64(f'{year}-{month}-{day}'))

In [ ]:
# vv.expand_dims({'band': 1})
darr_all.time.data

In [ ]:
arr_s2 = ds.evo_dataset._reader()._read_s2('11WMM_078_25_20200214')

In [ ]:
darr_merged = xr.concat(
            [arr_s2, darr_all.sel(time = darr_all.time.data[0])],
            dim="band",
            compat="equals",
        )

In [ ]:
darr_merged

In [ ]:
vv = darr.VV.data
vh = darr.VH.data
ratio = darr.VHonVV.data

In [ ]:
# # epsg = 4326 #src.crs.to_epsg()
from satio_pc.geotiff import compute_pixel_coordinates
bounds = arr_s2.satio.bounds
fill_value = -1
attrs = {"bounds": bounds}#"epsg": epsg, 
bands = ["VV", "VH"]
new_y, new_x = compute_pixel_coordinates(bounds, arr_s2.shape[-2:])
fill_value = -1
s1_arr = np.ones((len(bands), arr_s2.shape[-2], arr_s2.shape[-1])) * fill_value

darr_tst = xr.DataArray(
    s1_arr,
    dims=["band", "y", "x"],
    coords={"band": bands, "y": new_y, "x": new_x},
    attrs=attrs,
)

In [ ]:
darr_tst

In [ ]:
arr_s2

In [ ]:
# darr.VH.t
feats.shape

In [ ]:
darr.VV.t

In [ ]:
import rasterio
with rasterio.open() as src:
    

In [ ]:
list(darr.VV.t.data)[0]

In [ ]:
month = list(darr.VV.t.data)[0].astype('datetime64[M]').astype(int) % 12 + 1
month

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(vv[3,...])

In [ ]:
import torch

In [ ]:
t = torch.rand(64, 13, 256, 256)
m = torch.max(t, dim=1, keepdim=True)

In [ ]:
m[0]

In [ ]:
ds = EvoNetV1Dataset(
    bands=["B04", "B03", "B02", "cop-DEM-alt"],
    rgb_bands=None,
    flip_augmentation=True,
    rotate_augmentation=True,
    augmented_scaling=False,
    k_factor=5,
    k_factor_jitter=2,
    meteo_jitter=0.05,
    latlon_jitter=0.5,
    doy_jitter=15,
    shuffle_locs=False,
    sort_locs_by_latitude=False,
)

In [ ]:
len(ds.ids)

In [ ]:
len(ds.ids)

### Let's load some data

In [ ]:
import rasterio
with rasterio.open('/projects/TAP/vegteam/training_data/evotrain_v2tsm/annotations/2020/01/55/G/EN/evotrain_v2tsm_lsc-evo-v1-weight-cover-v1_55GEN_046_09_20200105.tif') as src:
    print(src.descriptions)

In [ ]:
it = 11
sample_id = ds.ids[it]
sample_id

In [ ]:
# len(ds)
it = 11
feats, feats_head, probs, weights = ds[it]
sample_id = ds.ids[it]

### Plot the loaded data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

nrows = 3
ncols = 7
fig = plt.figure(figsize=(10, 8))
gs = gridspec.GridSpec(nrows, ncols, figure=fig)

ax = fig.add_subplot(gs[0, 0])
plt.imshow(np.moveaxis(data.numpy()[:3], [0, 1, 2], [2, 0, 1]) * 2.5)
plt.title(f"SAMPLE ID {sample_id}")

for ind, prob in enumerate(probs.numpy()):
    # rgb
    r = (ind // ncols) + 1
    c = ((ind + 1) % ncols) - 1
    ax = fig.add_subplot(gs[r, c])
    plt.imshow(prob.squeeze())
    plt.title(f"Probability {NET_LABEL_NAMES[ind]}")


In [ ]:
data.shape

In [ ]:
head.shape

In [ ]:
probs.shape